In [ ]:
print(1)

In [7]:
import plistlib
import os
p1 = "defaults_before.plist"
print(os.path.exists(p1))
with open(p1, "r") as f:
    content = f.read()
    # Remove any non-ASCII characters that could cause issues
    content = ''.join(char for char in content if ord(char) < 128)
    # Ensure content starts with proper plist format
    if not content.strip().startswith('<?xml'):
        content = '<?xml version="1.0" encoding="UTF-8"?>\n<!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">\n<plist version="1.0">\n' + content + '\n</plist>'
with open(p1 + ".tmp", "w") as f:
    f.write(content)
d1 = plistlib.load(open(p1 + ".tmp", "rb"))

p2 = "defaults_after.plist"
with open(p2, "r") as f:
    content = f.read()
    content = ''.join(char for char in content if ord(char) < 128)
    if not content.strip().startswith('<?xml'):
        content = '<?xml version="1.0" encoding="UTF-8"?>\n<!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">\n<plist version="1.0">\n' + content + '\n</plist>'
with open(p2 + ".tmp", "w") as f:
    f.write(content)
d2 = plistlib.load(open(p2 + ".tmp", "rb"))

print(d1)
print(d2)


True


ExpatError: not well-formed (invalid token): line 6656, column 35

In [8]:
import subprocess
import plistlib

# Run defaults command and capture output
process = subprocess.run(['defaults', 'read'], capture_output=True, text=True)

# Write the output to a temporary file
with open('temp.plist', 'w') as f:
    f.write(process.stdout)

# Read the plist file
with open('temp.plist', 'rb') as f:
    data = plistlib.load(f)

InvalidFileException: Invalid file